In [1]:
!pip install selenium

In [2]:
import selenium
from selenium import webdriver
import urllib.parse
import time
from selenium.webdriver.common.keys import Keys
import os
import requests
import time
import pandas as pd

In [3]:
options = selenium.webdriver.ChromeOptions()
# options.add_argument('headless')
driver = webdriver.Chrome('/home/junior/Documents/PriceTrackerBot/python/googlePriceTracker/chromedriver', options=options)

In [4]:
url = "https://www.elo7.com.br/bellebiscuit"
projectPath = url.split('/')[-1]
main_window = driver.current_window_handle

In [5]:
driver.get(url)

In [6]:
def scroll_down():
    """A method for scrolling the page."""

    # Get scroll height.
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:

        # Scroll down to the bottom.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load the page.
        time.sleep(1)

        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:

            break

        last_height = new_height

#### Get produts

In [7]:
scroll_down()
productsList =  driver.find_element_by_class_name("product-list")
products = productsList.find_elements_by_class_name('store-product-item')

### Dive into the product and scrap

In [8]:
def collectProductDetails(driver, projectPath, photosFolder, productName, productLink):
    descHTML = ''
    productCode = ''
    descText = ''
    
    getPhotos(driver, photosFolder, productName)
    descHTML = driver.find_element_by_class_name('description').get_attribute('innerHTML')
    descText = driver.find_element_by_class_name('description').get_attribute('innerText')
    productCode = driver.find_element_by_class_name('detailed-info').find_element_by_tag_name('li').get_attribute('innerText').replace("Código do produto: ", "")
    return {"productDescHTML":descHTML, "productDescText":descText, "productCode":productCode}

### Loop through the thumbs and extract photos from the link

In [9]:
def getPhotos(driver, photosFolder, productName):
    thumbs = driver.find_element_by_class_name('thumbnails').find_elements_by_class_name('thumb')
    
    if not os.path.isdir(os.path.join(photosFolder, productName)):
        os.makedirs(photosFolder, exist_ok=True)
           
    
    print(f"Saving {len(thumbs)} photos of product **{productName}**...")
    for idx, thumb in enumerate(thumbs):
       
        photoLink = thumb.find_element_by_tag_name('link').get_attribute('href')
        photoName = photoLink.split('/')[-1]
        filename, ext = photoName.split(".")
        filename = f"{filename}_{str(idx).zfill(6)}.{ext}"
        
        if os.path.exists(os.path.join(photosFolder,filename)):
            print(f"File {os.path.join(photosFolder,filename)} already exists...")
            continue
               
        try:
            response = requests.get(photoLink)
            file = open(os.path.join(photosFolder,filename), "wb")
            file.write(response.content)
            file.close()
            time.sleep(0.5)
        except Exception as e:
            print(e)

In [10]:
parsedProductList = []
for idx, product in enumerate(products):
    figure = product.find_element_by_tag_name('figure')
    figCaption = figure.find_element_by_tag_name('figcaption')
    productLink = figCaption.find_element_by_tag_name('a').get_attribute('href')
    productName = figCaption.find_element_by_tag_name('a').get_attribute('innerText')
    productPrice = figCaption.find_elements_by_tag_name('span')[-1].get_attribute('innerText')
    
    photosFolder = os.path.join(projectPath, 'photos', productName)
    
    print("Diving into the product")
    driver.execute_script("window.open(''),'_blank'")  #open new tab
    driver.switch_to.window(driver.window_handles[1]) # Switch to the new window
    driver.get(productLink)
    time.sleep(2)
       
    productDetails = collectProductDetails(driver, projectPath, photosFolder, productName, productLink)   
    productDetails.update({"productName":productName, "productPrice":productPrice, "productLink":productLink})
    parsedProductList.append(productDetails)
    
    driver.close()
    driver.switch_to.window(main_window)
    time.sleep(2)
    pd.DataFrame([productDetails]).to_csv(os.path.join(projectPath,'products.csv'),index=False, mode='a', header=False)

Diving into the product
Saving 4 photos of product **Peter Pan Biscuit - Kit Festa**...
Diving into the product
Saving 5 photos of product **Toy Story - Cubos Decorados**...
Diving into the product
Saving 5 photos of product **Bonecas LOL - Caixinhas de Acrílico**...
Diving into the product
Saving 5 photos of product **Fundo do Mar- Mini potinho**...
Diving into the product
Saving 5 photos of product **Sereia - Vela decorada**...
Diving into the product
Saving 3 photos of product **PRONTA ENTREGA - Chuva de Amor - Baleiro**...
Diving into the product
Saving 3 photos of product **Aviador - Kit Festa**...
Diving into the product
Saving 5 photos of product **Os Vingadores - Mini Potinho**...
Diving into the product
Saving 4 photos of product **Unicórnio - Caixinhas de acrílico**...
Diving into the product
Saving 5 photos of product **Bonecas LOL - Garrafinhas**...
Diving into the product
Saving 3 photos of product **LadyBug e Cat Noir - Potinhos**...
Diving into the product
Saving 5 photo